# Main Runner

## Imports

In [ ]:
import pandas as pd
from objects import *
import pickle
import os
import matplotlib.pyplot as plt

## Generate Experiments

In [ ]:
prices = pd.read_parquet("raw_data/spx_stock_prices.parquet")
treasury_rate_files = ["daily-treasury-rates.csv"] + [f"daily-treasury-rates ({i}).csv" for i in range(1, 25)]
rates_df = [pd.read_csv(f"raw_data/{file}", index_col=0) for file in treasury_rate_files]
rates_df = pd.concat(rates_df)
rates_df.index = pd.to_datetime(rates_df.index)
# sort rates_df by date
rates_df = rates_df.sort_index()
print("Start Generation")
generate_experiments(prices, rates_df, 10, "experiments", lookback=48, error_max=9)

# Read in generated experiments

In [2]:
# Get all files in the experiments folder
experiment_names = os.listdir('experiments')
experiments = []
# read in pickle object
for e in experiment_names:
    with open(f"experiments/{e}", 'rb') as f:
        exp = pickle.load(f)
        experiments.append(exp)
exp = experiments[0]

## Evaluate Policies

In [3]:
multi_sim = MultiSimRunner(experiments, ["Naive", "DayTrading"])
multi_sim.run(save_file="naive_vs_daytrading.csv")

 40%|████      | 4/10 [1:05:44<1:38:36, 986.08s/it]


KeyboardInterrupt: 

In [ ]:
multi_sim.get_results()